In [1]:
"""
Add mypy type-checking cell magic to jupyter/ipython.
Save this script to your ipython profile's startup directory.
IPython's directories can be found via `ipython locate [profile]` to find the 
current ipython directory and ipython profile directory, respectively.
For example, this file could exist on a path like this on mac:
/Users/yourusername/.ipython/profile_default/startup/typecheck.py
where /Users/yourusername/.ipython/profile_default/ is the ipython directory for
the default profile.
The line magic is called "typecheck" to avoid namespace conflict with the mypy
package.
"""

from IPython.core.magic import register_cell_magic


@register_cell_magic
def typecheck(line, cell):
    """
    Run the following cell though mypy.
    Any parameters that would normally be passed to the mypy cli
    can be passed on the first line, with the exception of the
    -c flag we use to pass the code from the cell we want to execute
     i.e.
    %%typecheck --ignore-missing-imports
    ...
    ...
    ...
    mypy stdout and stderr will print prior to output of cell. If there are no conflicts,
    nothing will be printed by mypy.
    """

    from IPython import get_ipython
    from mypy import api

    # inserting a newline at the beginning of the cell
    # ensures mypy's output matches the the line
    # numbers in jupyter

    cell = '\n' + cell

    mypy_result = api.run(['-c', cell] + line.split())

    if mypy_result[0]:  # print mypy stdout
        print(mypy_result[0])

    if mypy_result[1]:  # print mypy stderr
        print(mypy_result[1])

    shell = get_ipython()
    shell.run_cell(cell)

In [2]:
import torch

# Verify that CUDA is properly configured and you have a GPU available# Verif 
torch.cuda.is_available()

True

In [3]:
%%typecheck --ignore-missing-imports

import itertools
import random
from typing import Any
from typing import List
from typing import Tuple

low: int = -100
high: int = 100
factors: List[Tuple[Any, ...]]  = [x for x in itertools.product(range(low, high), repeat=2)]
random.shuffle(factors)

In [4]:
%%typecheck --ignore-missing-imports

from typing import Any
from typing import List
from typing import Tuple
import torch
from torch.utils import data

class MultiplicationDataset(data.Dataset):
    '''Dataset for multiplication'''
    
    def __init__(self, factors: List[Tuple[Any, ...]]) -> None:
        self.factors: List[Tuple[Any, ...]] = factors
    
    def __len__(self) -> int:
        return len(self.factors)
    
    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        factors: torch.Tensor = torch.Tensor(self.factors[idx])
        return factors, factors[0] * factors[1]
    
    
def get_datasets(factors: List[Tuple[Any, ...]]) -> Tuple[data.Dataset, data.Dataset]:
    divider: int = int(len(factors) * 0.8)
    train_set: data.Dataset = MultiplicationDataset(factors[:divider])
    test_set: data.Dataset = MultiplicationDataset(factors[divider:])
    return train_set, test_set

In [5]:
batch_size: int = 40000
train_set, test_set = get_datasets(factors)

train_loader = data.DataLoader(dataset=train_set, batch_size=batch_size, 
                               shuffle=True, num_workers=4)
test_loader = data.DataLoader(dataset=test_set, batch_size=batch_size,
                              shuffle=False, num_workers=2)

In [6]:
%%typecheck --ignore-missing-imports

import os
import torch

def save_checkpoint(state, filename: str='checkpoint.pth.tar'):
    torch.save(state, filename)


def load_checkpoint(filename: str='checkpoint.pth.tar'):
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        return torch.load(filename)
    else:
        return None

In [7]:
import tensorflow as tf

def log_scalar_summary(writer, tag, value, step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
    writer.add_summary(summary, step)

In [8]:
%%typecheck --ignore-missing-imports

import torch
from torch import nn
from torch import autograd
from torch.utils import data

dtype = torch.cuda.FloatTensor

def check_accuracy(model: nn.Module, loader: data.DataLoader) -> float:
    model.eval()
    num_samples: int = 0
    num_correct: int = 0
    for x, y in loader:
        x_var: torch.Tensor = autograd.Variable(x.type(dtype), requires_grad=True)
        scores: torch.Tensor = model(x_var).view(-1)
        preds: torch.Tensor = torch.round(scores.data.cpu())
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    return float(num_correct) / num_samples

In [9]:
%%typecheck --ignore-missing-imports

import torch
from torch import nn
from torch import autograd
from torch.utils import data

dtype = torch.cuda.FloatTensor

def get_loss(model: nn.Module, loader: data.DataLoader, loss_fn: nn.Module) -> float:
    model.eval()
    total_loss: float = 0
    num_samples: int = 0
    for x, y in loader:
        x_var: torch.Tensor = autograd.Variable(x.type(dtype), volatile=True)
        y_var: torch.Tensor = y.type(dtype)
        # y_var: torch.Tensor = autograd.Variable(y.type(dtype))
        scores: torch.Tensor = model(x_var).view(-1)
        loss: torch.Tensor = loss_fn(scores, y_var)
        total_loss += loss.data[0].item()
        num_samples += y.shape[0]
    return total_loss / num_samples

In [10]:
%%typecheck --ignore-missing-imports

from typing import Iterable
import tensorflow as tf
import torch
from torch import autograd
from torch import nn
from torch import optim

dtype = torch.cuda.FloatTensor

def train(model: nn.Module, 
          loss_fn: nn.Module, 
          optimizer: optim.Optimizer,
          name: str,
          epochs: Iterable[int],
          gradient_clipping_norm: float=-1,
          epochs_per_save: int=20) -> None:
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                     mode='min',
                                                     factor=0.5, 
                                                     patience=10,
                                                     verbose=True)
    writer = tf.summary.FileWriter(name + '/logdir')
    for epoch in epochs:
        print('Starting epoch {}'.format(epoch + 1))
        total_loss: float = 0
        num_samples: int = 0
        model.train()
        for x, y in train_loader: # type: ignore
            x_var: torch.Tensor = autograd.Variable(x.type(dtype))
            y_var: torch.Tensor = autograd.Variable(y.type(dtype))
            scores: torch.Tensor = model(x_var).view(-1)
            loss: torch.Tensor = loss_fn(scores, y_var)
            total_loss += loss.data[0].item()
            num_samples += y.shape[0]
            optimizer.zero_grad()
            loss.backward()
            if gradient_clipping_norm > 0:
                nn.utils.clip_grad_norm_(model.parameters(), gradient_clipping_norm)
            optimizer.step()
        training_loss = total_loss / num_samples
        print('training loss:', training_loss)
        log_scalar_summary(writer, 'training loss', training_loss, epoch) # type: ignore
        test_acc = check_accuracy(model, test_loader) # type: ignore
        print('test accuracy:', test_acc)
        log_scalar_summary(writer, 'test accuracy', test_acc, epoch) # type: ignore
        test_loss = get_loss(model, test_loader, loss_fn) # type: ignore
        print('test loss:', test_loss)
        log_scalar_summary(writer, 'test loss', test_loss, epoch) # type: ignore
        
        if scheduler is not None:
            scheduler.step(training_loss)

        if (epoch + 1) % epochs_per_save == 0:
            save_checkpoint({ # type: ignore
                'epoch': epoch,
                'name': name,
                'state_dict': model.state_dict(),
                'acc': test_acc,
                'optimizer' : optimizer.state_dict(),
            }, '{}/{}-{}.pth.tar'.format(name, name, test_acc))

In [11]:
%%typecheck --ignore-missing-imports

from typing import List, Sequence
import torch
from torch import nn
from torchvision import models

class Pass(nn.Module):
    def forward(self, x):
        return x

dtype = torch.cuda.FloatTensor

def make_fc(layer_sizes: Sequence[int]=(2, 128, 128, 1),
            dropouts: Sequence[bool]=(False, False, False),
            relus: Sequence[bool]=(True, True, False),
            batchnorms: Sequence[bool]=(True, True, False)) -> nn.Module:
    sequential_layers: List[nn.Module] = []
    # Pair up consecutive layer sizes.
    # For ex, layer_sizes = [a, b, c] -> [(a, b), (b, c)]
    for (layer_size_in, layer_size_out), dropout, relu, batchnorm in zip(
            zip(layer_sizes[:-1], layer_sizes[1:]), dropouts, relus, batchnorms):
        sequential_layers.append(nn.Linear(layer_size_in, layer_size_out))
        if batchnorm:
            sequential_layers.append(nn.BatchNorm1d(layer_size_out))
        if relu:
            sequential_layers.append(nn.ReLU())
        if dropout:
            sequential_layers.append(nn.Dropout())
    model: nn.Module = nn.Sequential(*sequential_layers).type(dtype)
    return model

In [12]:
%%typecheck --ignore-missing-imports

import numpy as np
import torch
from torch import autograd
from torch import nn

dtype = torch.cuda.FloatTensor

model: nn.Module = make_fc( # type: ignore
    layer_sizes=(2, 128, 128, 128, 1),
    dropouts=(False, False, False, False),
    relus=(True, True, True, False),
    batchnorms=(True, True, True, False),
)
    
x_var: torch.Tensor = autograd.Variable(torch.randn(2, 2).type(dtype))
ans: torch.Tensor = model(x_var).view(-1)

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([2, ]))

True

In [13]:
loss_fn = nn.MSELoss(reduction='sum').type(dtype)

In [14]:
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0, weight_decay=5e-5)
train(model, loss_fn, optimizer, name='200fc3dropout', epochs=range(0, 8000), 
      gradient_clipping_norm=1000000, epochs_per_save=50)

Starting epoch 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss: 11098912.768
test accuracy: 0.0
test loss: 2795208704.0
Starting epoch 2


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


training loss: 11143414.784
test accuracy: 0.0
test loss: 76645151014.912
Starting epoch 3
training loss: 10821457.92
test accuracy: 0.0
test loss: 1952509581066.24
Starting epoch 4
training loss: 12021179.392
test accuracy: 0.0
test loss: 4036099833.856
Starting epoch 5
training loss: 11196565.504
test accuracy: 0.0
test loss: 4024173592.576
Starting epoch 6
training loss: 10507345.92
test accuracy: 0.0
test loss: 20372378353.664
Starting epoch 7
training loss: 9043531.776
test accuracy: 0.0
test loss: 71783164149.76
Starting epoch 8
training loss: 7295824.384
test accuracy: 0.0
test loss: 46082205679.616
Starting epoch 9
training loss: 9342603.264
test accuracy: 0.000125
test loss: 13465356.288
Starting epoch 10
training loss: 11273961.472
test accuracy: 0.0
test loss: 1535480692.736
Starting epoch 11
training loss: 7790254.08
test accuracy: 0.0
test loss: 5038272413.696
Starting epoch 12
training loss: 9571998.72
test accuracy: 0.0
test loss: 25949210.624
Starting epoch 13
training 

...

test accuracy: 0.02675
test loss: 384.740875
Starting epoch 7637
training loss: 315.561375
test accuracy: 0.025125
test loss: 374.119375
Starting epoch 7638
training loss: 314.02925
test accuracy: 0.026875
test loss: 384.7589375
Starting epoch 7639
training loss: 315.517125
test accuracy: 0.024875
test loss: 374.0273125
Starting epoch 7640
training loss: 314.00171875
test accuracy: 0.02725
test loss: 384.6698125
Starting epoch 7641
training loss: 315.46721875
test accuracy: 0.025125
test loss: 373.94509375
Starting epoch 7642
training loss: 313.948375
test accuracy: 0.027
test loss: 384.685875
Starting epoch 7643
training loss: 315.4465625
test accuracy: 0.02475
test loss: 373.86428125
Starting epoch 7644
training loss: 313.8533125
test accuracy: 0.027
test loss: 384.561875
Starting epoch 7645
training loss: 315.385375
test accuracy: 0.024875
test loss: 373.8150625
Starting epoch 7646
training loss: 313.84771875
test accuracy: 0.027
test loss: 384.61296875
Starting epoch 7647
training 

test accuracy: 0.024875
test loss: 372.121375
Starting epoch 7726
training loss: 312.0780625
test accuracy: 0.0255
test loss: 382.890625
Starting epoch 7727
training loss: 313.55534375
test accuracy: 0.025125
test loss: 372.0475625
Starting epoch 7728
training loss: 312.03175
test accuracy: 0.025375
test loss: 382.8249375
Starting epoch 7729
training loss: 313.508375
test accuracy: 0.025125
test loss: 371.9576875
Starting epoch 7730
training loss: 311.9861875
test accuracy: 0.0255
test loss: 382.7888125
Starting epoch 7731
training loss: 313.4876875
test accuracy: 0.02525
test loss: 371.9309375
Starting epoch 7732
training loss: 311.92375
test accuracy: 0.025375
test loss: 382.793125
Starting epoch 7733
training loss: 313.4179375
test accuracy: 0.025125
test loss: 371.9144375
Starting epoch 7734
training loss: 311.8873125
test accuracy: 0.025375
test loss: 382.78925
Starting epoch 7735
training loss: 313.38078125
test accuracy: 0.02525
test loss: 371.880125
Starting epoch 7736
training

training loss: 310.13815625
test accuracy: 0.025125
test loss: 381.1439375
Starting epoch 7815
training loss: 311.63171875
test accuracy: 0.025375
test loss: 370.083125
Starting epoch 7816
training loss: 310.1191875
test accuracy: 0.0255
test loss: 381.126375
Starting epoch 7817
training loss: 311.5814375
test accuracy: 0.025
test loss: 370.0045
Starting epoch 7818
training loss: 310.0443125
test accuracy: 0.025125
test loss: 381.120375
Starting epoch 7819
training loss: 311.53153125
test accuracy: 0.024875
test loss: 369.88803125
Starting epoch 7820
training loss: 309.99925
test accuracy: 0.02575
test loss: 381.108
Starting epoch 7821
training loss: 311.4668125
test accuracy: 0.025125
test loss: 369.7921875
Starting epoch 7822
training loss: 309.96584375
test accuracy: 0.02575
test loss: 381.0895625
Starting epoch 7823
training loss: 311.42840625
test accuracy: 0.025
test loss: 369.67834375
Starting epoch 7824
training loss: 309.90665625
test accuracy: 0.025875
test loss: 381.063375
S

training loss: 309.638
test accuracy: 0.023625
test loss: 367.8083125
Starting epoch 7904
training loss: 308.18275
test accuracy: 0.025125
test loss: 379.485625
Starting epoch 7905
training loss: 309.60478125
test accuracy: 0.023875
test loss: 367.67615625
Starting epoch 7906
training loss: 308.13578125
test accuracy: 0.025125
test loss: 379.43971875
Starting epoch 7907
training loss: 309.5685625
test accuracy: 0.024125
test loss: 367.638125
Starting epoch 7908
training loss: 308.10746875
test accuracy: 0.025
test loss: 379.4176875
Starting epoch 7909
training loss: 309.5200625
test accuracy: 0.024
test loss: 367.577125
Starting epoch 7910
training loss: 308.07425
test accuracy: 0.025
test loss: 379.3513125
Starting epoch 7911
training loss: 309.48746875
test accuracy: 0.02425
test loss: 367.4909375
Starting epoch 7912
training loss: 308.0285
test accuracy: 0.025
test loss: 379.377
Starting epoch 7913
training loss: 309.438
test accuracy: 0.02425
test loss: 367.442625
Starting epoch 79

training loss: 306.2639375
test accuracy: 0.024625
test loss: 377.43175
Starting epoch 7993
training loss: 307.67684375
test accuracy: 0.02325
test loss: 365.6711875
Starting epoch 7994
training loss: 306.24196875
test accuracy: 0.024375
test loss: 377.4113125
Starting epoch 7995
training loss: 307.6213125
test accuracy: 0.023375
test loss: 365.62075
Starting epoch 7996
training loss: 306.1923125
test accuracy: 0.0245
test loss: 377.39690625
Starting epoch 7997
training loss: 307.58371875
test accuracy: 0.02325
test loss: 365.6248125
Starting epoch 7998
training loss: 306.11959375
test accuracy: 0.024375
test loss: 377.388875
Starting epoch 7999
training loss: 307.55125
test accuracy: 0.023375
test loss: 365.4890625
Starting epoch 8000
training loss: 306.08615625
test accuracy: 0.024375
test loss: 377.3139375
